In [11]:
import os
import sys
import csv
import random
import itertools
from operator import itemgetter
from collections import defaultdict
import numpy as np
import scipy
import scipy.spatial.distance
from numpy.linalg import svd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import utils

In [18]:
numDimen = 300
gloveFile = 'glove.6B.' + str(numDimen) + 'd.txt'
GLOVE = utils.glove2dict(os.path.join('glove.6B', gloveFile))

In [19]:
def randvec(n=50, lower=-0.5, upper=0.5):
    """Returns a random vector of length `n`"""
    return np.array([random.uniform(lower, upper) for i in range(n)])

def getGloveVec(w):    
    """Return `w`'s GloVe representation if available, else return 
    a random vector."""
    return GLOVE.get(w, randvec(n=numDimen))

def getCosineSim(u, v):        
    """Cosine distance between 1d np.arrays `u` and `v`, which must have 
    the same dimensionality. Returns a float."""
    return scipy.spatial.distance.cosine(u, v)

print getCosineSim(getGloveVec('the'), getGloveVec('hello'))

0.985473873557


In [20]:
def sentenceToGlove(words):
    result = [0] * numDimen
    for word in words:
        result = np.add(result, getGloveVec(word))
    return result
    

    
print sentenceToGlove(['the','hello','PewDiePiePewDiePie']) == sentenceToGlove(['the','hello'])


[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa